Установка одинаковых версий

In [ ]:
!pip install -q keras==2.15.0

In [ ]:
!pip install -q tensorflow==2.15.0

Импорт библиотек

In [ ]:
import keras
import tensorflow
print(keras.__version__)
print(tensorflow.__version__)
import pandas as pd
import numpy as np
import re
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import Input, Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

2.15.0
2.15.0


Загрузка данных

In [ ]:
df = pd.read_csv('no_NaN_geometry_sakhalin.csv')
df = df[df['date'] == '2020-06-01.tif'][:2000]

In [ ]:
df['sr4_3_mean'] = df['B08_mean'] / df['B04_mean']
df['sr5_4_mean'] = df['B11_mean'] / df['B08_mean']
df['kt_1_mean'] = (0.3037 * df['B02_mean'] + 0.2793 * df['B03_mean'] +
                   0.4743 * df['B04_mean'] + 0.5585 * df['B08_mean'] +
                   0.5082 * df['B11_mean'] + 0.1863 * df['B12_mean'])
df['kt_2_mean'] = (-0.2941 * df['B02_mean'] - 0.243 * df['B03_mean'] -
                   0.5424 * df['B04_mean'] + 0.7276 * df['B08_mean'] +
                   0.0713 * df['B11_mean'] - 0.1608 * df['B12_mean'])
df['kt_3_mean'] = (0.1511 * df['B02_mean'] + 0.1973 * df['B03_mean'] +
                   0.3283 * df['B04_mean'] + 0.3407 * df['B08_mean'] -
                   0.7117 * df['B11_mean'] - 0.4559 * df['B12_mean'])


In [ ]:
def avg_long(polygon):
    my_string = polygon
    my_string = re.sub(r'[A-Za-z()]', '', my_string)
    longitudes = [float(coord.split(' ')[1]) for coord in my_string.split(',')]
    return np.mean(longitudes)

def avg_lat(polygon):
    my_string = polygon
    my_string = re.sub(r'[A-Za-z()]', '', my_string)
    latitudes = [float(coord.split(' ')[2]) for coord in my_string.split(',')]
    return np.mean(latitudes)

In [ ]:
df['mean_latitude'] = df['geometry'].apply(avg_lat)
df['mean_longitude'] = df['geometry'].apply(avg_long)

Столбцы в данных, разделение признаков на целевые и переменные

In [ ]:
y_height = df['height']
y_timber_stock = df['timber_stock']
y_basal_area = df['basal_area']

In [ ]:
df.drop(['date', 'height', 'timber_stock', 'basal_area', 'geometry'], axis=1, inplace=True)
X = df

Деление данных на обучающую и тестоовые выборки

In [ ]:
X_train, X_test, y_height_train, y_height_test, y_timber_stock_train, y_timber_stock_test, y_basal_area_train, y_basal_area_test = train_test_split(
    X, y_height, y_timber_stock, y_basal_area, test_size=0.2, random_state=42)

масштабирование выборки

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

создание модели

In [ ]:
input_layer = Input(shape=(X_train_scaled.shape[1],))

20 общих слоев

In [ ]:
shared_layer = input_layer
for _ in range(10):  # 10 слоев с 256 нейронами
    shared_layer = Dense(256, activation='relu')(shared_layer)
    shared_layer = Dropout(0.2)(shared_layer)

for _ in range(10):  # 10 слоев с 128 нейронами
    shared_layer = Dense(128, activation='relu')(shared_layer)
    shared_layer = Dropout(0.2)(shared_layer)

ветви высоты, древисины и площади

In [ ]:
height_output = Dense(1, name='height_output')(shared_layer)
timber_stock_output = Dense(1, name='timber_stock_output')(shared_layer)
basal_area_output = Dense(1, name='basal_area_output')(shared_layer)

создание модели

In [ ]:
model = Model(inputs=input_layer, outputs=[height_output, timber_stock_output, basal_area_output])

компиляция модели

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

бучение модели

In [ ]:
history = model.fit(X_train_scaled, [y_height_train, y_timber_stock_train, y_basal_area_train],
                    validation_data=(X_test_scaled, [y_height_test, y_timber_stock_test, y_basal_area_test]),
                    epochs=100, batch_size=32)

Epoch 1/100
119/119 [==============================] - 18s 50ms/step - loss: 11291.0654 - height_output_loss: 208.9468 - timber_stock_output_loss: 10908.3379 - basal_area_output_loss: 173.7808 - height_output_mae: 11.1593 - timber_stock_output_mae: 79.1721 - basal_area_output_mae: 9.6430 - val_loss: 11114.3584 - val_height_output_loss: 101.7169 - val_timber_stock_output_loss: 10971.6025 - val_basal_area_output_loss: 41.0387 - val_height_output_mae: 9.7331 - val_timber_stock_output_mae: 78.1237 - val_basal_area_output_mae: 5.1866
Epoch 2/100
119/119 [==============================] - 3s 25ms/step - loss: 8126.1777 - height_output_loss: 109.6872 - timber_stock_output_loss: 7929.1382 - basal_area_output_loss: 87.3514 - height_output_mae: 8.1123 - timber_stock_output_mae: 67.2929 - basal_area_output_mae: 6.9707 - val_loss: 8690.6094 - val_height_output_loss: 30.8080 - val_timber_stock_output_loss: 8638.6084 - val_basal_area_output_loss: 21.1937 - val_height_output_mae: 4.9607 - val_timber_

Оценка модели

In [ ]:
loss, height_loss, timber_stock_loss, basal_area_loss, height_mae, timber_stock_mae, basal_area_mae = model.evaluate(X_test_scaled, [y_height_test, y_timber_stock_test, y_basal_area_test])

30/30 [==============================] - 0s 5ms/step - loss: 4963.6543 - height_output_loss: 7.3358 - timber_stock_output_loss: 4943.5020 - basal_area_output_loss: 12.8174 - height_output_mae: 2.0016 - timber_stock_output_mae: 54.2926 - basal_area_output_mae: 2.7562


In [ ]:
y_pred = model.predict(X_test_scaled)
y_pred_height = y_pred[0].ravel()
y_pred_timber_stock = y_pred[1].ravel()
y_pred_basal_area = y_pred[2].ravel()

30/30 [==============================] - 0s 5ms/step


In [ ]:
r_sq_height = r2_score(y_height_test, y_pred_height)
mape_height = np.mean(np.abs((y_height_test - y_pred_height) / y_height_test)) * 100
mae_height = mean_absolute_error(y_height_test, y_pred_height)

In [ ]:
r_sq_timber_stock = r2_score(y_timber_stock_test, y_pred_timber_stock)
mape_timber_stock = np.mean(np.abs((y_timber_stock_test - y_pred_timber_stock) / y_timber_stock_test)) * 100
mae_timber_stock = mean_absolute_error(y_timber_stock_test, y_pred_timber_stock)

In [ ]:
r_sq_basal_area = r2_score(y_basal_area_test, y_pred_basal_area)
mape_basal_area = np.mean(np.abs((y_basal_area_test - y_pred_basal_area) / y_basal_area_test)) * 100
mae_basal_area = mean_absolute_error(y_basal_area_test, y_pred_basal_area)

In [ ]:
print(f'Coefficient of determination (height): {r_sq_height}')
print(f'MAPE (height): {mape_height}')
print(f'MAE (height): {mae_height}')
print(f'MSE (height): {mean_squared_error(y_height_test, y_pred_height)}\n')

print(f'Coefficient of determination (timber stock): {r_sq_timber_stock}')
print(f'MAPE (timber stock): {mape_timber_stock}')
print(f'MAE (timber stock): {mae_timber_stock}')
print(f'MSE (timber stock): {mean_squared_error(y_timber_stock_test, y_pred_timber_stock)}\n')

print(f'Coefficient of determination (basal area): {r_sq_basal_area}')
print(f'MAPE (basal area): {mape_basal_area}')
print(f'MAE (basal area): {mae_basal_area}')
print(f'MSE (basal area): {mean_squared_error(y_basal_area_test, y_pred_basal_area)}\n')


Coefficient of determination (height): 0.1076692797318517
MAPE (height): 16.355892097068597
MAE (height): 2.0016480339001905
MSE (height): 7.335814769860439

Coefficient of determination (timber stock): 0.2910347141545454
MAPE (timber stock): 56.76249552167884
MAE (timber stock): 54.29263734716694
MSE (timber stock): 4943.501360015251

Coefficient of determination (basal area): 0.2743216726919264
MAPE (basal area): 36.24183246946035
MAE (basal area): 2.756155158684735
MSE (basal area): 12.817389503754153



Добавление XGB

In [ ]:
model_xgb_height = xgb.XGBRegressor(objective='reg:squarederror')
model_xgb_height.fit(X_train_scaled, y_height_train)

model_xgb_timber_stock = xgb.XGBRegressor(objective='reg:squarederror')
model_xgb_timber_stock.fit(X_train_scaled, y_timber_stock_train)

model_xgb_basal_area = xgb.XGBRegressor(objective='reg:squarederror')
model_xgb_basal_area.fit(X_train_scaled, y_basal_area_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [ ]:
y_pred_xgb_height = model_xgb_height.predict(X_test_scaled)
y_pred_xgb_timber_stock = model_xgb_timber_stock.predict(X_test_scaled)
y_pred_xgb_basal_area = model_xgb_basal_area.predict(X_test_scaled)

In [ ]:
r_sq_xgb_height = r2_score(y_height_test, y_pred_xgb_height)
mae_xgb_height = mean_absolute_error(y_height_test, y_pred_xgb_height)

r_sq_xgb_timber_stock = r2_score(y_timber_stock_test, y_pred_xgb_timber_stock)
mae_xgb_timber_stock = mean_absolute_error(y_timber_stock_test, y_pred_xgb_timber_stock)

r_sq_xgb_basal_area = r2_score(y_basal_area_test, y_pred_xgb_basal_area)
mae_xgb_basal_area = mean_absolute_error(y_basal_area_test, y_pred_xgb_basal_area)

In [ ]:
print(f'XGBoost - Coefficient of determination (height): {r_sq_xgb_height}')
print(f'XGBoost - MAE (height): {mae_xgb_height}\n')

print(f'XGBoost - Coefficient of determination (timber stock): {r_sq_xgb_timber_stock}')
print(f'XGBoost - MAE (timber stock): {mae_xgb_timber_stock}\n')

print(f'XGBoost - Coefficient of determination (basal area): {r_sq_xgb_basal_area}')
print(f'XGBoost - MAE (basal area): {mae_xgb_basal_area}\n')

XGBoost - Coefficient of determination (height): 0.28917858685572695
XGBoost - MAE (height): 1.745328742908373

XGBoost - Coefficient of determination (timber stock): 0.2882941757536218
XGBoost - MAE (timber stock): 53.92739856873204

XGBoost - Coefficient of determination (basal area): 0.22743074147644793
XGBoost - MAE (basal area): 2.821859195733187

